In [ ]:
!pip install -q torchvision==0.15.2 torch==2.2.0 albumentations==1.3.1

ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3
ERROR: Could not find a version that satisfies the requirement torchvision==0.15.2 (from versions: 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.24.0, 0.24.1)
ERROR: No matching distribution found for torchvision==0.15.2


In [ ]:
import os
import zipfile
from pathlib import Path

ZIP_PATH = "/content/Labeled Data.zip"
DATA_ROOT = "/content/labeled_data"

if not os.path.exists(DATA_ROOT):
    print("Unzipping dataset...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(DATA_ROOT)
else:
    print("Dataset already extracted to", DATA_ROOT)

Unzipping dataset...


In [ ]:
print("\nTop-level folders (classes):")
print(sorted([p.name for p in Path(DATA_ROOT).iterdir() if p.is_dir()])[:50])


Top-level folders (classes):
['Labeled Data']


In [ ]:
import random
import time
from collections import Counter
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
# Cell 3 - custom enhancement to help low-quality faces

from PIL import Image
import numpy as np

class EnhanceLowQuality:
    """
    A PIL -> PIL transform that applies CLAHE on the Y channel (better contrast)
    and light denoising using OpenCV.
    """
    def __init__(self, clipLimit=3.0, tileGridSize=(8,8), denoise_strength=3):
        self.clipLimit = clipLimit
        self.tileGridSize = tileGridSize
        self.denoise_strength = denoise_strength

    def __call__(self, img: Image.Image):
        # convert PIL to BGR numpy (uint8)
        arr = np.array(img)
        if arr.ndim == 2:  # grayscale
            arr_bgr = cv2.cvtColor(arr, cv2.COLOR_GRAY2BGR)
        else:
            arr_bgr = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)

        # Convert to YCrCb and apply CLAHE to Y channel
        ycrcb = cv2.cvtColor(arr_bgr, cv2.COLOR_BGR2YCrCb)
        y, cr, cb = cv2.split(ycrcb)
        clahe = cv2.createCLAHE(clipLimit=self.clipLimit, tileGridSize=self.tileGridSize)
        y_eq = clahe.apply(y)
        ycrcb = cv2.merge([y_eq, cr, cb])
        enhanced = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

        # Light denoise
        enhanced = cv2.fastNlMeansDenoisingColored(enhanced, None, h=self.denoise_strength, hColor=self.denoise_strength, templateWindowSize=7, searchWindowSize=21)

        # convert back to PIL RGB
        enhanced_rgb = cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB)
        return Image.fromarray(enhanced_rgb)


In [ ]:
import os
import pathlib

root = "/content/labeled_data"   # or whatever folder you extracted to

print("Checking folder:", root)
for item in pathlib.Path(root).rglob("*"):
    print(item)


Checking folder: /content/labeled_data
/content/labeled_data/Labeled Data
/content/labeled_data/Labeled Data/maghol
/content/labeled_data/Labeled Data/Marwan
/content/labeled_data/Labeled Data/Hania Khaled
/content/labeled_data/Labeled Data/maryam
/content/labeled_data/Labeled Data/Habiba Ayman
/content/labeled_data/Labeled Data/Doha
/content/labeled_data/Labeled Data/shahd
/content/labeled_data/Labeled Data/Abdallah Tamer
/content/labeled_data/Labeled Data/Hamza
/content/labeled_data/Labeled Data/Norhan
/content/labeled_data/Labeled Data/Menna Omar
/content/labeled_data/Labeled Data/Maryam EL-Haje
/content/labeled_data/Labeled Data/Raihana
/content/labeled_data/Labeled Data/Tokka
/content/labeled_data/Labeled Data/Habib
/content/labeled_data/Labeled Data/kareem
/content/labeled_data/Labeled Data/Mark
/content/labeled_data/Labeled Data/Hager Ali
/content/labeled_data/Labeled Data/radwa
/content/labeled_data/Labeled Data/Ahmed Amer
/content/labeled_data/Labeled Data/Salma Wael
/content/

In [ ]:
import os

root = "/content/labeled_data/Labeled Data"

for person in sorted(os.listdir(root)):
    person_path = os.path.join(root, person)
    if os.path.isdir(person_path):
        count = len([f for f in os.listdir(person_path) if f.lower().endswith((".jpg", ".png"))])
        print(f"{person:20s} --> {count} images")


Abdallah Tamer       --> 49 images
Ahmed Amer           --> 15 images
Ameen                --> 43 images
Anas                 --> 37 images
Bayomy               --> 20 images
Doha                 --> 1 images
Habib                --> 31 images
Habiba Ayman         --> 41 images
Hager Ali            --> 43 images
Hamza                --> 47 images
Hania Khaled         --> 51 images
Kahla                --> 37 images
Mark                 --> 42 images
Marwan               --> 30 images
Maryam EL-Haje       --> 40 images
Menna Omar           --> 38 images
Mohamed              --> 40 images
Norhan               --> 54 images
Nosyba               --> 31 images
Ossama               --> 34 images
Raihana              --> 17 images
Roaa                 --> 25 images
Rofida               --> 25 images
Salma Wael           --> 52 images
Tokka                --> 39 images
kareem               --> 21 images
maghol               --> 39 images
maghol 2             --> 15 images
maryam               

In [ ]:
import cv2

root = "/content/labeled_data/Labeled Data"

for person in sorted(os.listdir(root)):
    person_path = os.path.join(root, person)
    for img in os.listdir(person_path):
        img_path = os.path.join(person_path, img)
        im = cv2.imread(img_path)
        if im is None:
            print("CORRUPT:", img_path)


In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder("/content/labeled_data/Labeled Data", transform=transform)


In [ ]:
import os
import shutil
import random

for person in os.listdir(DATA_ROOT):
    folder = os.path.join(DATA_ROOT, person)
    imgs = [f for f in os.listdir(folder) if f.lower().endswith(("jpg","png"))]

    if len(imgs) == 1:
        print("Duplicating images for:", person)
        src = os.path.join(folder, imgs[0])
        for i in range(2):   # make 2 extra copies (now total = 3)
            shutil.copy(src, os.path.join(folder, f"copy_{i}.jpg"))


In [ ]:
import shutil

DATA_ROOT = "/content/labeled_data/Labeled Data"

# Remove .ipynb_checkpoints
chk = os.path.join(DATA_ROOT, ".ipynb_checkpoints")
if os.path.exists(chk):
    print("Removing .ipynb_checkpoints...")
    shutil.rmtree(chk)

# Remove any empty folders
for person in os.listdir(DATA_ROOT):
    folder = os.path.join(DATA_ROOT, person)
    if os.path.isdir(folder):
        imgs = [f for f in os.listdir(folder) if f.lower().endswith(("jpg","png","jpeg"))]
        if len(imgs) == 0:
            print("Removing empty folder:", folder)
            shutil.rmtree(folder)

print("Cleaned folder structure!\nNow loading dataset...")


Removing .ipynb_checkpoints...
Cleaned folder structure!
Now loading dataset...


In [ ]:
from sklearn.model_selection import train_test_split
import shutil

DATA_ROOT = "/content/labeled_data/Labeled Data"

all_imgs = datasets.ImageFolder(DATA_ROOT).imgs
all_paths = [p for p, l in all_imgs]
all_labels = [l for p, l in all_imgs]

print("Total images:", len(all_imgs))
print("Classes:", sorted(os.listdir(DATA_ROOT)))

train_paths, val_paths = train_test_split(
    all_imgs, test_size=0.2, stratify=all_labels, random_state=42
)

# recreate folders
!rm -rf /content/train /content/val
!mkdir /content/train /content/val

def rebuild(paths, dest):
    for p, label in paths:
        class_name = os.path.basename(os.path.dirname(p))
        class_folder = os.path.join(dest, class_name)
        os.makedirs(class_folder, exist_ok=True)
        shutil.copy(p, class_folder)

rebuild(train_paths, "/content/train")
rebuild(val_paths, "/content/val")

print("Train and Val folders created!")


Total images: 1107
Classes: ['Abdallah Tamer', 'Ahmed Amer', 'Ameen', 'Anas', 'Bayomy', 'Habib', 'Habiba Ayman', 'Hager Ali', 'Hamza', 'Hania Khaled', 'Kahla', 'Mark', 'Marwan', 'Maryam EL-Haje', 'Menna Omar', 'Mohamed', 'Norhan', 'Nosyba', 'Ossama', 'Raihana', 'Roaa', 'Rofida', 'Salma Wael', 'Tokka', 'kareem', 'maghol', 'maghol 2', 'maryam', 'radwa', 'shahd']
Train and Val folders created!


In [ ]:
from PIL import Image

class SiameseDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None, enhance=False):
        self.dataset = datasets.ImageFolder(root)
        self.transform = transform
        self.enhance = enhance
        self.enhancer = EnhanceLowQuality()

        self.images = self.dataset.imgs
        self.class_to_images = {}

        for path, label in self.images:
            self.class_to_images.setdefault(label, []).append(path)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img1_path, label = self.images[idx]
        img1 = Image.open(img1_path).convert("RGB")

        # optional enhancement
        if self.enhance:
            img1 = self.enhancer(img1)

        same = random.choice([0, 1])

        if same == 1:
            img2_path = random.choice(self.class_to_images[label])
        else:
            diff_label = random.choice([l for l in self.class_to_images.keys() if l != label])
            img2_path = random.choice(self.class_to_images[diff_label])

        img2 = Image.open(img2_path).convert("RGB")

        if self.enhance:
            img2 = self.enhancer(img2)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor([same], dtype=torch.float32)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_ds = SiameseDataset("/content/train", transform, enhance=True)
val_ds   = SiameseDataset("/content/val",   transform, enhance=True)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))


Train batches: 56
Val batches: 14


In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        base = models.mobilenet_v2(weights="IMAGENET1K_V2")
        self.feature = base.features
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1280, 256)

    def forward_once(self, x):
        x = self.feature(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return F.normalize(x)

    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)


In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, out1, out2, label):
        dist = F.pairwise_distance(out1, out2)
        loss = torch.mean(
            label * dist.pow(2) +
            (1 - label) * torch.clamp(self.margin - dist, min=0).pow(2)
        )
        return loss


In [ ]:
import torch.nn.functional as F

In [ ]:
model = SiameseNetwork().to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for img1, img2, label in train_loader:
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)

        out1, out2 = model(img1, img2)
        loss = criterion(out1, out2, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {train_loss/len(train_loader):.4f}")


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 64.3MB/s]


Epoch 1/10 - Loss: 0.2604
Epoch 2/10 - Loss: 0.2534
Epoch 3/10 - Loss: 0.2563
Epoch 4/10 - Loss: 0.2549
Epoch 5/10 - Loss: 0.2559
Epoch 6/10 - Loss: 0.2555
Epoch 7/10 - Loss: 0.2563
Epoch 8/10 - Loss: 0.2571
Epoch 9/10 - Loss: 0.2565
Epoch 10/10 - Loss: 0.2544


In [ ]:
torch.save(model.state_dict(), "/content/siamese_face_model.pth")
print("Model saved!")


Model saved!


In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load("/content/siamese_face_model.pth", map_location=device))
model.eval()

print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
def get_embedding(img_path):
    img = Image.open(img_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = model.forward_once(img)
    return emb


In [ ]:
REFERENCE_ROOT = "/content/train"

reference_embeddings = {}

for person in sorted(os.listdir(REFERENCE_ROOT)):
    folder = os.path.join(REFERENCE_ROOT, person)
    imgs = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(("jpg","png","jpeg"))]

    if len(imgs) == 0:
        continue

    ref_img = imgs[0]
    reference_embeddings[person] = get_embedding(ref_img)

    print(f"Reference built for: {person}")


Reference built for: Abdallah Tamer
Reference built for: Ahmed Amer
Reference built for: Ameen
Reference built for: Anas
Reference built for: Bayomy
Reference built for: Habib
Reference built for: Habiba Ayman
Reference built for: Hager Ali
Reference built for: Hamza
Reference built for: Hania Khaled
Reference built for: Kahla
Reference built for: Mark
Reference built for: Marwan
Reference built for: Maryam EL-Haje
Reference built for: Menna Omar
Reference built for: Mohamed
Reference built for: Norhan
Reference built for: Nosyba
Reference built for: Ossama
Reference built for: Raihana
Reference built for: Roaa
Reference built for: Rofida
Reference built for: Salma Wael
Reference built for: Tokka
Reference built for: kareem
Reference built for: maghol
Reference built for: maghol 2
Reference built for: maryam
Reference built for: radwa
Reference built for: shahd


In [ ]:
def predict_person(img_path):
    query_emb = get_embedding(img_path)

    sims = {}
    for name, ref_emb in reference_embeddings.items():
        sim = F.cosine_similarity(query_emb, ref_emb).item()
        sims[name] = sim

    best = max(sims, key=sims.get)

    print("=== Similarity Scores ===")
    for n, s in sims.items():
        print(f"{n}: {s:.4f}")

    print("\nPredicted Identity:", best)
    return best, sims


In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_004_conf_1.00.jpg to face_004_conf_1.00.jpg
Predicting for: face_004_conf_1.00.jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8512
Ahmed Amer: 0.8385
Ameen: 0.9636
Anas: 0.8888
Bayomy: 0.9171
Habib: 0.8342
Habiba Ayman: 0.8911
Hager Ali: 0.9039
Hamza: 0.8897
Hania Khaled: 0.8160
Kahla: 0.8852
Mark: 0.8459
Marwan: 0.8926
Maryam EL-Haje: 0.9112
Menna Omar: 0.8456
Mohamed: 0.8183
Norhan: 0.9054
Nosyba: 0.8467
Ossama: 0.8782
Raihana: 0.8727
Roaa: 0.8211
Rofida: 0.8983
Salma Wael: 0.9110
Tokka: 0.8767
kareem: 0.8878
maghol: 0.8356
maghol 2: 0.8753
maryam: 0.8189
radwa: 0.8802
shahd: 0.8804

Predicted Identity: Ameen


('Ameen',
 {'Abdallah Tamer': 0.8511823415756226,
  'Ahmed Amer': 0.8384679555892944,
  'Ameen': 0.9636436104774475,
  'Anas': 0.8887742161750793,
  'Bayomy': 0.9170591831207275,
  'Habib': 0.8341882228851318,
  'Habiba Ayman': 0.891132652759552,
  'Hager Ali': 0.9038905501365662,
  'Hamza': 0.8896595239639282,
  'Hania Khaled': 0.8159698843955994,
  'Kahla': 0.8851795792579651,
  'Mark': 0.8458970785140991,
  'Marwan': 0.8925639390945435,
  'Maryam EL-Haje': 0.9111982583999634,
  'Menna Omar': 0.8455514311790466,
  'Mohamed': 0.8183003664016724,
  'Norhan': 0.9054376482963562,
  'Nosyba': 0.8467317223548889,
  'Ossama': 0.8781710863113403,
  'Raihana': 0.8727145791053772,
  'Roaa': 0.8210502862930298,
  'Rofida': 0.8983330130577087,
  'Salma Wael': 0.9110202789306641,
  'Tokka': 0.8766865730285645,
  'kareem': 0.8877986669540405,
  'maghol': 0.8355615139007568,
  'maghol 2': 0.8752739429473877,
  'maryam': 0.8188750147819519,
  'radwa': 0.8801676630973816,
  'shahd': 0.880385279655456

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_006_conf_1.00 (2).jpg to face_006_conf_1.00 (2).jpg
Predicting for: face_006_conf_1.00 (2).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8385
Ahmed Amer: 0.8512
Ameen: 0.9169
Anas: 0.8511
Bayomy: 0.8917
Habib: 0.8157
Habiba Ayman: 0.8738
Hager Ali: 0.9174
Hamza: 0.8480
Hania Khaled: 0.7818
Kahla: 0.9082
Mark: 0.7972
Marwan: 0.8942
Maryam EL-Haje: 0.9221
Menna Omar: 0.8217
Mohamed: 0.7522
Norhan: 0.9280
Nosyba: 0.8391
Ossama: 0.8678
Raihana: 0.8683
Roaa: 0.7907
Rofida: 0.8455
Salma Wael: 0.9170
Tokka: 0.8564
kareem: 0.8490
maghol: 0.8367
maghol 2: 0.8753
maryam: 0.7875
radwa: 0.8910
shahd: 0.8510

Predicted Identity: Norhan


('Norhan',
 {'Abdallah Tamer': 0.8385413885116577,
  'Ahmed Amer': 0.8512184619903564,
  'Ameen': 0.9169279336929321,
  'Anas': 0.8511192202568054,
  'Bayomy': 0.8916606903076172,
  'Habib': 0.8156751394271851,
  'Habiba Ayman': 0.8737842440605164,
  'Hager Ali': 0.9174000024795532,
  'Hamza': 0.848022997379303,
  'Hania Khaled': 0.7817585468292236,
  'Kahla': 0.9081636667251587,
  'Mark': 0.7971515655517578,
  'Marwan': 0.8942068815231323,
  'Maryam EL-Haje': 0.9221392869949341,
  'Menna Omar': 0.8217076063156128,
  'Mohamed': 0.7521974444389343,
  'Norhan': 0.9280195832252502,
  'Nosyba': 0.8390694260597229,
  'Ossama': 0.8678065538406372,
  'Raihana': 0.8682776689529419,
  'Roaa': 0.7906887531280518,
  'Rofida': 0.8455170392990112,
  'Salma Wael': 0.9170458316802979,
  'Tokka': 0.8564280271530151,
  'kareem': 0.8489829897880554,
  'maghol': 0.8367477059364319,
  'maghol 2': 0.8753381967544556,
  'maryam': 0.7874782085418701,
  'radwa': 0.8909512162208557,
  'shahd': 0.85104984045028

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_009_conf_1.00 (2).jpg to face_009_conf_1.00 (2).jpg
Predicting for: face_009_conf_1.00 (2).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8349
Ahmed Amer: 0.9101
Ameen: 0.8966
Anas: 0.8566
Bayomy: 0.9198
Habib: 0.8791
Habiba Ayman: 0.8740
Hager Ali: 0.9052
Hamza: 0.8880
Hania Khaled: 0.8314
Kahla: 0.9053
Mark: 0.8506
Marwan: 0.9255
Maryam EL-Haje: 0.8923
Menna Omar: 0.8677
Mohamed: 0.8158
Norhan: 0.8826
Nosyba: 0.8761
Ossama: 0.9020
Raihana: 0.9280
Roaa: 0.8467
Rofida: 0.8705
Salma Wael: 0.9011
Tokka: 0.8847
kareem: 0.8824
maghol: 0.9351
maghol 2: 0.9532
maryam: 0.7923
radwa: 0.8809
shahd: 0.8967

Predicted Identity: maghol 2


('maghol 2',
 {'Abdallah Tamer': 0.8349360227584839,
  'Ahmed Amer': 0.9100890159606934,
  'Ameen': 0.896557629108429,
  'Anas': 0.8565906882286072,
  'Bayomy': 0.9197548031806946,
  'Habib': 0.8790982961654663,
  'Habiba Ayman': 0.8739892840385437,
  'Hager Ali': 0.905217170715332,
  'Hamza': 0.8879591226577759,
  'Hania Khaled': 0.8314045071601868,
  'Kahla': 0.9053092002868652,
  'Mark': 0.8506243824958801,
  'Marwan': 0.9254616498947144,
  'Maryam EL-Haje': 0.8922634720802307,
  'Menna Omar': 0.8677448034286499,
  'Mohamed': 0.8158359527587891,
  'Norhan': 0.8825799822807312,
  'Nosyba': 0.8761016726493835,
  'Ossama': 0.9020214080810547,
  'Raihana': 0.9280097484588623,
  'Roaa': 0.846674382686615,
  'Rofida': 0.8705070614814758,
  'Salma Wael': 0.9011166095733643,
  'Tokka': 0.884746253490448,
  'kareem': 0.8824109435081482,
  'maghol': 0.9350897073745728,
  'maghol 2': 0.953211784362793,
  'maryam': 0.7923089861869812,
  'radwa': 0.8809412121772766,
  'shahd': 0.8966996073722839

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_006_conf_1.00 (4).jpg to face_006_conf_1.00 (4).jpg
Predicting for: face_006_conf_1.00 (4).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8665
Ahmed Amer: 0.8635
Ameen: 0.8754
Anas: 0.8663
Bayomy: 0.9096
Habib: 0.9134
Habiba Ayman: 0.9226
Hager Ali: 0.8920
Hamza: 0.9210
Hania Khaled: 0.8917
Kahla: 0.8004
Mark: 0.9028
Marwan: 0.8763
Maryam EL-Haje: 0.8792
Menna Omar: 0.9029
Mohamed: 0.8768
Norhan: 0.8845
Nosyba: 0.8736
Ossama: 0.8842
Raihana: 0.8654
Roaa: 0.9151
Rofida: 0.9361
Salma Wael: 0.8970
Tokka: 0.9155
kareem: 0.8924
maghol: 0.8569
maghol 2: 0.8904
maryam: 0.8605
radwa: 0.8944
shahd: 0.9103

Predicted Identity: Rofida


('Rofida',
 {'Abdallah Tamer': 0.8665409088134766,
  'Ahmed Amer': 0.8635374307632446,
  'Ameen': 0.8754018545150757,
  'Anas': 0.8662586808204651,
  'Bayomy': 0.9095948338508606,
  'Habib': 0.9133656620979309,
  'Habiba Ayman': 0.9225553274154663,
  'Hager Ali': 0.8919879794120789,
  'Hamza': 0.9210470914840698,
  'Hania Khaled': 0.8917413949966431,
  'Kahla': 0.8003880977630615,
  'Mark': 0.9028204679489136,
  'Marwan': 0.8762537240982056,
  'Maryam EL-Haje': 0.8791747689247131,
  'Menna Omar': 0.9029136300086975,
  'Mohamed': 0.8767790794372559,
  'Norhan': 0.8844675421714783,
  'Nosyba': 0.8736150860786438,
  'Ossama': 0.8842041492462158,
  'Raihana': 0.8654120564460754,
  'Roaa': 0.9150789976119995,
  'Rofida': 0.9360800981521606,
  'Salma Wael': 0.8969500064849854,
  'Tokka': 0.9155368208885193,
  'kareem': 0.8924044966697693,
  'maghol': 0.8569121360778809,
  'maghol 2': 0.8904350399971008,
  'maryam': 0.8605208992958069,
  'radwa': 0.894429624080658,
  'shahd': 0.91025966405868

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_012_conf_1.00 (3).jpg to face_012_conf_1.00 (3).jpg
Predicting for: face_012_conf_1.00 (3).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.9225
Ahmed Amer: 0.9173
Ameen: 0.8693
Anas: 0.9316
Bayomy: 0.9066
Habib: 0.8994
Habiba Ayman: 0.9217
Hager Ali: 0.9384
Hamza: 0.9007
Hania Khaled: 0.8740
Kahla: 0.8918
Mark: 0.8711
Marwan: 0.8957
Maryam EL-Haje: 0.9153
Menna Omar: 0.9072
Mohamed: 0.8663
Norhan: 0.9087
Nosyba: 0.9321
Ossama: 0.9467
Raihana: 0.8953
Roaa: 0.9118
Rofida: 0.8831
Salma Wael: 0.8780
Tokka: 0.9099
kareem: 0.9311
maghol: 0.9033
maghol 2: 0.9180
maryam: 0.8593
radwa: 0.9223
shahd: 0.9350

Predicted Identity: Ossama


('Ossama',
 {'Abdallah Tamer': 0.9225496649742126,
  'Ahmed Amer': 0.9172742962837219,
  'Ameen': 0.8693200349807739,
  'Anas': 0.9315570592880249,
  'Bayomy': 0.9065913558006287,
  'Habib': 0.8994339108467102,
  'Habiba Ayman': 0.9216918349266052,
  'Hager Ali': 0.9384355545043945,
  'Hamza': 0.9006633162498474,
  'Hania Khaled': 0.8740448951721191,
  'Kahla': 0.8918370008468628,
  'Mark': 0.8711026906967163,
  'Marwan': 0.8956818580627441,
  'Maryam EL-Haje': 0.9152984619140625,
  'Menna Omar': 0.9071749448776245,
  'Mohamed': 0.8662905693054199,
  'Norhan': 0.9086925387382507,
  'Nosyba': 0.9321200251579285,
  'Ossama': 0.946742594242096,
  'Raihana': 0.895334005355835,
  'Roaa': 0.9118451476097107,
  'Rofida': 0.8831284046173096,
  'Salma Wael': 0.8780337572097778,
  'Tokka': 0.9099114537239075,
  'kareem': 0.931128978729248,
  'maghol': 0.9032577872276306,
  'maghol 2': 0.9180020093917847,
  'maryam': 0.8592816591262817,
  'radwa': 0.9223191738128662,
  'shahd': 0.9350405931472778

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving 22.jpg to 22.jpg
Predicting for: 22.jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8350
Ahmed Amer: 0.8787
Ameen: 0.9214
Anas: 0.8881
Bayomy: 0.9230
Habib: 0.8564
Habiba Ayman: 0.8768
Hager Ali: 0.9216
Hamza: 0.9154
Hania Khaled: 0.8324
Kahla: 0.8831
Mark: 0.8334
Marwan: 0.8974
Maryam EL-Haje: 0.9141
Menna Omar: 0.8524
Mohamed: 0.8099
Norhan: 0.9217
Nosyba: 0.8660
Ossama: 0.8979
Raihana: 0.8793
Roaa: 0.8417
Rofida: 0.9104
Salma Wael: 0.9143
Tokka: 0.8931
kareem: 0.9511
maghol: 0.8655
maghol 2: 0.9031
maryam: 0.8181
radwa: 0.8938
shahd: 0.8852

Predicted Identity: kareem


('kareem',
 {'Abdallah Tamer': 0.8349970579147339,
  'Ahmed Amer': 0.8787376284599304,
  'Ameen': 0.9214057326316833,
  'Anas': 0.8880929946899414,
  'Bayomy': 0.9230443239212036,
  'Habib': 0.8564472794532776,
  'Habiba Ayman': 0.8768444657325745,
  'Hager Ali': 0.921554684638977,
  'Hamza': 0.9154486656188965,
  'Hania Khaled': 0.8324060440063477,
  'Kahla': 0.8831046223640442,
  'Mark': 0.8333522081375122,
  'Marwan': 0.8973556756973267,
  'Maryam EL-Haje': 0.9141430854797363,
  'Menna Omar': 0.8524332046508789,
  'Mohamed': 0.8098727464675903,
  'Norhan': 0.9216920137405396,
  'Nosyba': 0.8660476207733154,
  'Ossama': 0.8978598117828369,
  'Raihana': 0.8792821168899536,
  'Roaa': 0.8417049050331116,
  'Rofida': 0.9104390740394592,
  'Salma Wael': 0.9143452048301697,
  'Tokka': 0.8930670619010925,
  'kareem': 0.9510622620582581,
  'maghol': 0.865474283695221,
  'maghol 2': 0.9030718207359314,
  'maryam': 0.8181135058403015,
  'radwa': 0.8937757015228271,
  'shahd': 0.885186791419982

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_010_conf_1.00 (3).jpg to face_010_conf_1.00 (3).jpg
Predicting for: face_010_conf_1.00 (3).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8828
Ahmed Amer: 0.8796
Ameen: 0.8421
Anas: 0.8759
Bayomy: 0.8929
Habib: 0.9389
Habiba Ayman: 0.8988
Hager Ali: 0.8861
Hamza: 0.9176
Hania Khaled: 0.9083
Kahla: 0.7946
Mark: 0.9173
Marwan: 0.8893
Maryam EL-Haje: 0.8684
Menna Omar: 0.9021
Mohamed: 0.8999
Norhan: 0.8649
Nosyba: 0.9029
Ossama: 0.9088
Raihana: 0.8825
Roaa: 0.9193
Rofida: 0.8939
Salma Wael: 0.8591
Tokka: 0.9176
kareem: 0.8917
maghol: 0.8764
maghol 2: 0.8861
maryam: 0.8443
radwa: 0.8823
shahd: 0.9204

Predicted Identity: Habib


('Habib',
 {'Abdallah Tamer': 0.8827716112136841,
  'Ahmed Amer': 0.8795923590660095,
  'Ameen': 0.842105507850647,
  'Anas': 0.8759425282478333,
  'Bayomy': 0.8928850889205933,
  'Habib': 0.9389142990112305,
  'Habiba Ayman': 0.8988033533096313,
  'Hager Ali': 0.8861150741577148,
  'Hamza': 0.9176002740859985,
  'Hania Khaled': 0.9083338975906372,
  'Kahla': 0.7946053147315979,
  'Mark': 0.9172922372817993,
  'Marwan': 0.8893241286277771,
  'Maryam EL-Haje': 0.8684438467025757,
  'Menna Omar': 0.9021045565605164,
  'Mohamed': 0.8998758792877197,
  'Norhan': 0.8649464845657349,
  'Nosyba': 0.9029266834259033,
  'Ossama': 0.9087539911270142,
  'Raihana': 0.882518470287323,
  'Roaa': 0.9192948341369629,
  'Rofida': 0.893899142742157,
  'Salma Wael': 0.859119713306427,
  'Tokka': 0.9176143407821655,
  'kareem': 0.8916537761688232,
  'maghol': 0.8763808608055115,
  'maghol 2': 0.8860583305358887,
  'maryam': 0.8442769050598145,
  'radwa': 0.8823296427726746,
  'shahd': 0.9204478859901428})

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_006_conf_1.00 (2).jpg to face_006_conf_1.00 (2) (1).jpg
Predicting for: face_006_conf_1.00 (2) (1).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.9196
Ahmed Amer: 0.8629
Ameen: 0.8865
Anas: 0.9186
Bayomy: 0.8915
Habib: 0.8812
Habiba Ayman: 0.9145
Hager Ali: 0.9163
Hamza: 0.9088
Hania Khaled: 0.8615
Kahla: 0.8631
Mark: 0.8809
Marwan: 0.8956
Maryam EL-Haje: 0.9278
Menna Omar: 0.9256
Mohamed: 0.8793
Norhan: 0.9237
Nosyba: 0.9025
Ossama: 0.9156
Raihana: 0.8679
Roaa: 0.8868
Rofida: 0.9089
Salma Wael: 0.9153
Tokka: 0.9178
kareem: 0.9058
maghol: 0.8485
maghol 2: 0.8821
maryam: 0.8562
radwa: 0.9167
shahd: 0.9095

Predicted Identity: Maryam EL-Haje


('Maryam EL-Haje',
 {'Abdallah Tamer': 0.9195541143417358,
  'Ahmed Amer': 0.862907886505127,
  'Ameen': 0.886487603187561,
  'Anas': 0.9186416864395142,
  'Bayomy': 0.8914518356323242,
  'Habib': 0.8812248706817627,
  'Habiba Ayman': 0.9144579768180847,
  'Hager Ali': 0.9163455963134766,
  'Hamza': 0.9088023900985718,
  'Hania Khaled': 0.8614785075187683,
  'Kahla': 0.8631273508071899,
  'Mark': 0.8808972835540771,
  'Marwan': 0.8956420421600342,
  'Maryam EL-Haje': 0.927775502204895,
  'Menna Omar': 0.9256396889686584,
  'Mohamed': 0.8793085813522339,
  'Norhan': 0.9236944913864136,
  'Nosyba': 0.9025467038154602,
  'Ossama': 0.9156370162963867,
  'Raihana': 0.8679096698760986,
  'Roaa': 0.8867853283882141,
  'Rofida': 0.9088628888130188,
  'Salma Wael': 0.9153168201446533,
  'Tokka': 0.9177839159965515,
  'kareem': 0.9057692289352417,
  'maghol': 0.8485380411148071,
  'maghol 2': 0.8821051716804504,
  'maryam': 0.8562099933624268,
  'radwa': 0.9166553020477295,
  'shahd': 0.90946042

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_005_conf_1.00.jpg to face_005_conf_1.00.jpg
Predicting for: face_005_conf_1.00.jpg
=== Similarity Scores ===
Abdallah Tamer: 0.9000
Ahmed Amer: 0.8872
Ameen: 0.8724
Anas: 0.8982
Bayomy: 0.9087
Habib: 0.8841
Habiba Ayman: 0.8989
Hager Ali: 0.9384
Hamza: 0.8853
Hania Khaled: 0.8401
Kahla: 0.8657
Mark: 0.8440
Marwan: 0.9019
Maryam EL-Haje: 0.8982
Menna Omar: 0.8873
Mohamed: 0.8437
Norhan: 0.8936
Nosyba: 0.9009
Ossama: 0.9147
Raihana: 0.9106
Roaa: 0.9024
Rofida: 0.8743
Salma Wael: 0.8898
Tokka: 0.8912
kareem: 0.9193
maghol: 0.8831
maghol 2: 0.9018
maryam: 0.8168
radwa: 0.9017
shahd: 0.9054

Predicted Identity: Hager Ali


('Hager Ali',
 {'Abdallah Tamer': 0.8999536037445068,
  'Ahmed Amer': 0.8872323632240295,
  'Ameen': 0.8723952174186707,
  'Anas': 0.8982025384902954,
  'Bayomy': 0.9087109565734863,
  'Habib': 0.8840596675872803,
  'Habiba Ayman': 0.8988950848579407,
  'Hager Ali': 0.9384455680847168,
  'Hamza': 0.8852976560592651,
  'Hania Khaled': 0.840144157409668,
  'Kahla': 0.8656526803970337,
  'Mark': 0.8440151214599609,
  'Marwan': 0.9019472002983093,
  'Maryam EL-Haje': 0.8981605768203735,
  'Menna Omar': 0.8873156309127808,
  'Mohamed': 0.8437069654464722,
  'Norhan': 0.8935950994491577,
  'Nosyba': 0.9009294509887695,
  'Ossama': 0.914691150188446,
  'Raihana': 0.9105933308601379,
  'Roaa': 0.9024360179901123,
  'Rofida': 0.8743306398391724,
  'Salma Wael': 0.8897899389266968,
  'Tokka': 0.891170859336853,
  'kareem': 0.9193274974822998,
  'maghol': 0.8830575942993164,
  'maghol 2': 0.9017937779426575,
  'maryam': 0.8168277740478516,
  'radwa': 0.9017259478569031,
  'shahd': 0.9054430723190

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_008_conf_1.00.jpg to face_008_conf_1.00.jpg
Predicting for: face_008_conf_1.00.jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8932
Ahmed Amer: 0.8973
Ameen: 0.8469
Anas: 0.8540
Bayomy: 0.8903
Habib: 0.9134
Habiba Ayman: 0.8840
Hager Ali: 0.8847
Hamza: 0.9277
Hania Khaled: 0.8885
Kahla: 0.8148
Mark: 0.9148
Marwan: 0.9009
Maryam EL-Haje: 0.8808
Menna Omar: 0.9140
Mohamed: 0.8889
Norhan: 0.8701
Nosyba: 0.8940
Ossama: 0.9015
Raihana: 0.8893
Roaa: 0.9144
Rofida: 0.8956
Salma Wael: 0.8676
Tokka: 0.8876
kareem: 0.8818
maghol: 0.8782
maghol 2: 0.8988
maryam: 0.8386
radwa: 0.8684
shahd: 0.9040

Predicted Identity: Hamza


('Hamza',
 {'Abdallah Tamer': 0.8931671380996704,
  'Ahmed Amer': 0.8973342776298523,
  'Ameen': 0.8468986749649048,
  'Anas': 0.8539947271347046,
  'Bayomy': 0.8902819156646729,
  'Habib': 0.9133764505386353,
  'Habiba Ayman': 0.8840303421020508,
  'Hager Ali': 0.8846985697746277,
  'Hamza': 0.9277082681655884,
  'Hania Khaled': 0.8885415196418762,
  'Kahla': 0.8148152828216553,
  'Mark': 0.9147509336471558,
  'Marwan': 0.9008607864379883,
  'Maryam EL-Haje': 0.8807966709136963,
  'Menna Omar': 0.9140029549598694,
  'Mohamed': 0.8889175057411194,
  'Norhan': 0.8701306581497192,
  'Nosyba': 0.8940324783325195,
  'Ossama': 0.9015105962753296,
  'Raihana': 0.8893491625785828,
  'Roaa': 0.9143754243850708,
  'Rofida': 0.8955667018890381,
  'Salma Wael': 0.8675978779792786,
  'Tokka': 0.8875551223754883,
  'kareem': 0.881782591342926,
  'maghol': 0.8782016038894653,
  'maghol 2': 0.8987910151481628,
  'maryam': 0.8385534286499023,
  'radwa': 0.8683851957321167,
  'shahd': 0.904031872749328

In [ ]:
from google.colab import files
uploaded = files.upload()

test_img = list(uploaded.keys())[0]
print("Predicting for:", test_img)

predict_person(test_img)


Saving face_005_conf_1.00 (4).jpg to face_005_conf_1.00 (4).jpg
Predicting for: face_005_conf_1.00 (4).jpg
=== Similarity Scores ===
Abdallah Tamer: 0.8849
Ahmed Amer: 0.8529
Ameen: 0.8203
Anas: 0.8244
Bayomy: 0.8666
Habib: 0.9133
Habiba Ayman: 0.9051
Hager Ali: 0.8444
Hamza: 0.8873
Hania Khaled: 0.9142
Kahla: 0.7557
Mark: 0.9346
Marwan: 0.8504
Maryam EL-Haje: 0.8504
Menna Omar: 0.8857
Mohamed: 0.9020
Norhan: 0.8515
Nosyba: 0.8692
Ossama: 0.8769
Raihana: 0.8449
Roaa: 0.8834
Rofida: 0.8682
Salma Wael: 0.8224
Tokka: 0.8918
kareem: 0.8246
maghol: 0.8202
maghol 2: 0.8401
maryam: 0.8767
radwa: 0.8688
shahd: 0.9003

Predicted Identity: Mark


('Mark',
 {'Abdallah Tamer': 0.884934663772583,
  'Ahmed Amer': 0.8529285788536072,
  'Ameen': 0.8203104138374329,
  'Anas': 0.8243861198425293,
  'Bayomy': 0.8666182160377502,
  'Habib': 0.9133341312408447,
  'Habiba Ayman': 0.9050828218460083,
  'Hager Ali': 0.8444266319274902,
  'Hamza': 0.8872750401496887,
  'Hania Khaled': 0.9142288565635681,
  'Kahla': 0.7556577920913696,
  'Mark': 0.9346128702163696,
  'Marwan': 0.8504084348678589,
  'Maryam EL-Haje': 0.8503924608230591,
  'Menna Omar': 0.885658860206604,
  'Mohamed': 0.9020315408706665,
  'Norhan': 0.8514755964279175,
  'Nosyba': 0.8692009449005127,
  'Ossama': 0.8769254684448242,
  'Raihana': 0.8448735475540161,
  'Roaa': 0.8833605647087097,
  'Rofida': 0.8681603074073792,
  'Salma Wael': 0.8224325776100159,
  'Tokka': 0.8917763233184814,
  'kareem': 0.8245912790298462,
  'maghol': 0.820158839225769,
  'maghol 2': 0.840125322341919,
  'maryam': 0.8766614198684692,
  'radwa': 0.868781566619873,
  'shahd': 0.9002688527107239})

In [41]:
# ============================
#   VALIDATION ACCURACY CELL
# ============================

from sklearn.metrics import accuracy_score
import torch.nn.functional as F

print("\nComputing validation accuracy...\n")

val_root = "/content/val"
y_true = []
y_pred = []

model.eval()

for person in sorted(os.listdir(val_root)):
    person_folder = os.path.join(val_root, person)
    images = [os.path.join(person_folder, f)
              for f in os.listdir(person_folder)
              if f.lower().endswith(("jpg", "png", "jpeg"))]

    for img_path in images:
        pred, _ = predict_person(img_path)   # your function already returns best person
        y_true.append(person)
        y_pred.append(pred)

acc = accuracy_score(y_true, y_pred)

print("===================================")
print(f"Validation Accuracy: {acc * 100:.2f}%")
print("===================================")


Streaming output truncated to the last 5000 lines.
Ossama: 0.8941
Raihana: 0.8790
Roaa: 0.9298
Rofida: 0.8871
Salma Wael: 0.8539
Tokka: 0.9166
kareem: 0.8640
maghol: 0.8514
maghol 2: 0.8669
maryam: 0.8598
radwa: 0.8884
shahd: 0.9179

Predicted Identity: Roaa
=== Similarity Scores ===
Abdallah Tamer: 0.8350
Ahmed Amer: 0.8787
Ameen: 0.9214
Anas: 0.8881
Bayomy: 0.9230
Habib: 0.8564
Habiba Ayman: 0.8768
Hager Ali: 0.9216
Hamza: 0.9154
Hania Khaled: 0.8324
Kahla: 0.8831
Mark: 0.8334
Marwan: 0.8974
Maryam EL-Haje: 0.9141
Menna Omar: 0.8524
Mohamed: 0.8099
Norhan: 0.9217
Nosyba: 0.8660
Ossama: 0.8979
Raihana: 0.8793
Roaa: 0.8417
Rofida: 0.9104
Salma Wael: 0.9143
Tokka: 0.8931
kareem: 0.9511
maghol: 0.8655
maghol 2: 0.9031
maryam: 0.8181
radwa: 0.8938
shahd: 0.8852

Predicted Identity: kareem
=== Similarity Scores ===
Abdallah Tamer: 0.9155
Ahmed Amer: 0.8595
Ameen: 0.8902
Anas: 0.8759
Bayomy: 0.9096
Habib: 0.8885
Habiba Ayman: 0.9272
Hager Ali: 0.8846
Hamza: 0.9067
Hania Khaled: 0.8775
Kahla

In [42]:


model.eval()
correct = 0
total = 0
threshold = 0.5

with torch.no_grad():
    for img1, img2, label in val_loader:
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        out1, out2 = model(img1, img2)
        dist = F.pairwise_distance(out1, out2)
        preds = (dist < threshold).float()
        correct += (preds == label).sum().item()
        total += label.size(0)

accuracy = correct / total
print("===================================")
print(f"Siamese Model Accuracy on Validation Set: {accuracy*100:.2f}%")
print("===================================")


Siamese Model Accuracy on Validation Set: 760.36%
